In [6]:
# %matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pickle
from umap import UMAP
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

In [7]:
# load data
epoch = 100
N = 5000 # image number
y_train_path = './ResNetVAE-2024/plotly_umap_example/y_cocpit_training_hand_labeled_noaug_bw_resnet18_e100_b50_l3_z128_train_epoch100.npy' # edit path accordingly
z_train_path = './ResNetVAE-2024/plotly_umap_example/z_cocpit_training_hand_labeled_noaug_bw_resnet18_e100_b50_l3_z128_train_epoch100.npy' # edit path accordingly
y_train = np.load(y_train_path)
z_train = np.load(z_train_path)
classes = np.arange(9)

# Reduce to 3-d and create animation

In [8]:
# turn data into pandas df
data = np.column_stack((z_train, y_train))
df = pd.DataFrame(data)
n_cols = df.shape[1]
features = df.loc[:, 0:(n_cols-2)]
col_names = []
for i in range(n_cols-1):
    col_names.append(f'feature_{i}')
col_names.append('habit')
df.columns = col_names
idx_to_class = {0:'agg', 1:'budding', 2:'bullet', 3:'column', 4:'compact_irreg', 5:'fragment', 6:'planar_polycrystal', 7:'rimed', 8:'sphere'}
df.habit = df.habit.replace(idx_to_class)

umap_3d = UMAP(n_components=3, init='spectral', random_state=0)
proj_3d = umap_3d.fit_transform(features)
fig_3d = px.scatter_3d(
    proj_3d, x=0, y=1, z=2,
    color=df.habit, labels={'color': 'habit'}
)
fig_3d.update_traces(marker_size=5)
x_eye = 1.25
y_eye = 1.25
z_eye = 0.75
fig_3d.update_layout(
    autosize=False,
    width=800,
    height=600,
    scene = dict(
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        zaxis =dict(visible=False)
    ),
    scene_camera_eye=dict(x=x_eye, y=y_eye, z=z_eye),
    updatemenus=[dict(type='buttons',
                showactive=False,
                y=1,
                x=0.8,
                xanchor='left',
                yanchor='bottom',
                pad=dict(t=45, r=10),
                buttons=[dict(label='Play',
                        method='animate',
                        args=[None, dict(frame=dict(duration=100, redraw=True), 
                                        transition=dict(duration=0),
                                        fromcurrent=True,
                                        mode='immediate')])])
                ]
)
fig_3d.update_traces(marker_size=2.5, opacity=0.8)
def rotate_z(x, y, z, theta):
    w = x+1j*y
    return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z
frames=[]
for t in np.arange(0, 6.26, 0.1):
    xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
    frames.append(go.Frame(layout=dict(scene_camera_eye=dict(x=xe, y=ye, z=ze))))
fig_3d.frames=frames
fig_3d.show()

/home/jko/miniconda3/envs/umap/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.



# Reduce to 2D and plot

In [9]:
umap_2d = UMAP(n_components=2, init='random', random_state=0)
proj_2d = umap_2d.fit_transform(features)

fig_2d = px.scatter(
    proj_2d, x=0, y=1,
    width = 800, height = 600, 
    color=df.habit, labels={'color': 'habit'}
)
fig_2d.update_traces(marker_size=4, opacity=0.5)
fig_2d.show()

/home/jko/miniconda3/envs/umap/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning:

n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.

